In [1]:
import os
import torch
import yaml
from pathlib import Path
import random
import matplotlib.pyplot as plt
import numpy as np
import tqdm
import plotly.graph_objects as go
from torch.utils.data import Dataset

from nerfstudio.configs import base_config as cfg
from nerfstudio.configs.method_configs import method_configs
from nerfstudio.data.dataparsers.nerfosr_dataparser import NeRFOSR, NeRFOSRDataParserConfig
from nerfstudio.pipelines.base_pipeline import VanillaDataManager
from nerfstudio.field_components.field_heads import FieldHeadNames
from nerfstudio.cameras.rays import RayBundle, RaySamples, Frustums
from nerfstudio.utils.colormaps import apply_depth_colormap
from nerfstudio.field_components.encodings import SHEncoding, NeRFEncoding
import tinycudann as tcnn

from reni_neus.models.reni_neus_model import RENINeuSFactoModelConfig, RENINeuSFactoModel
from reni_neus.utils.utils import look_at_target, random_points_on_unit_sphere
from reni_neus.data.datamanagers.reni_neus_datamanager import RENINeuSDataManagerConfig, RENINeuSDataManager
from reni_neus.configs.ddf_config import DirectionalDistanceField
from reni_neus.configs.reni_neus_config import RENINeuS
from reni_neus.utils.utils import find_nerfstudio_project_root, rot_z

project_root = find_nerfstudio_project_root(Path(os.getcwd()))
# set current working directory to nerfstudio project root
os.chdir(project_root)

# setup config
test_mode = 'test'
world_size = 1
local_rank = 0
device = 'cuda:0'

reni_neus_config = RENINeuS
reni_neus_config.config.pipeline.visibility_ckpt_path = Path('outputs/ddf/ddf/2023-08-31_101658/')
reni_neus_config.config.pipeline.visibility_ckpt_step = 20000
reni_neus_config.config.pipeline.reni_neus_ckpt_path = Path('outputs/reni-neus/reni-neus/2023-08-30_111340/')
reni_neus_config.config.pipeline.reni_neus_ckpt_step = 100000
reni_neus_config.config.pipeline.model.use_visibility = True
reni_neus_config.config.pipeline.model.visibility_threshold = 0.1
reni_neus_config.config.pipeline.model.fit_visibility_field = False


pipeline = reni_neus_config.config.pipeline.setup(device=device, test_mode=test_mode, world_size=world_size, local_rank=local_rank)
datamanager = pipeline.datamanager
model = pipeline.model
model = model.eval()

Setting up training dataset...

Caching all 160 images.

Output()

Setting up evaluation dataset...

Caching all 95 images.

Output()

Caching all 95 images.

Output()

Caching all 95 images.

Output()

In [16]:
ray_bundle, batch = datamanager.get_nerfosr_lighting_eval_bundle(stage='optimise')
indices_to_session = datamanager.indices_to_session
# need to update batch['indices'][:, 0] using indices_to_session which is a dict of keys: indices (ints) and values: session (ints)
batch['indices'][:, 0] = torch.tensor([indices_to_session[i.item()] for i in batch['indices'][:, 0]]).type_as(batch['indices'][:, 0])

type

In [19]:
batch['indices']

tensor([[   1,  470,  240],
        [   4,  552,  896],
        [   2,  637,  207],
        [   3,  810,  390],
        [   1,  744,  368],
        [   2,  398,  841],
        [   2,  810,  602],
        [   2,  179, 1153],
        [   3,  306,  505],
        [   2,   17,  162],
        [   0,  193,  970],
        [   0,  596,  240],
        [   0,  220,   30],
        [   3,  158,  524],
        [   4,  632,  494],
        [   0,   51,  504],
        [   4,  301,  839],
        [   2,  307,  480],
        [   4,   57,  169],
        [   3,  244,  192],
        [   3,  583,  749],
        [   4,  726,  842],
        [   2,  776,  282],
        [   1,  494,  152],
        [   2,  787, 1176],
        [   1,  798,  732],
        [   0,  546, 1062],
        [   3,  167,   29],
        [   1,  497,  606],
        [   2,  496,  648],
        [   4,  284,  939],
        [   3,  172,  792],
        [   0,  448,  478],
        [   1,  285,  755],
        [   1,  341,  726],
        [   3,  625,

In [11]:
# remap indices [:, 0] using indices_to_session
batch['indices'][:, 0] 

tensor([60,  0, 60, 15, 31,  0, 15,  0, 31, 82,  0, 31,  0, 60,  0, 60, 82,  0,
         0, 82,  0, 15, 15, 31, 82, 82,  0, 82, 31, 60, 31,  0, 15, 15,  0, 15,
        82, 15,  0, 31, 82, 82, 31, 60, 60, 60,  0, 15, 31, 15, 31,  0, 60, 15,
        15,  0, 15, 31, 15, 82, 82, 82, 31, 31], device='cuda:0')

In [6]:
ray_bundle.camera_indices

tensor([[60],
        [ 0],
        [60],
        [15],
        [31],
        [ 0],
        [15],
        [ 0],
        [31],
        [82],
        [ 0],
        [31],
        [ 0],
        [60],
        [ 0],
        [60],
        [82],
        [ 0],
        [ 0],
        [82],
        [ 0],
        [15],
        [15],
        [31],
        [82],
        [82],
        [ 0],
        [82],
        [31],
        [60],
        [31],
        [ 0],
        [15],
        [15],
        [ 0],
        [15],
        [82],
        [15],
        [ 0],
        [31],
        [82],
        [82],
        [31],
        [60],
        [60],
        [60],
        [ 0],
        [15],
        [31],
        [15],
        [31],
        [ 0],
        [60],
        [15],
        [15],
        [ 0],
        [15],
        [31],
        [15],
        [82],
        [82],
        [82],
        [31],
        [31]], device='cuda:0')